### 1. Perform exact inference
Treat each column as one variable with 2^n states and perform message passing on the induced factor-graph.

In [2]:
import numpy as np
from itertools import product

# Ising model parameters
N = 10  # Lattice size (10x10)
beta_values = [4, 3, 1, 0.5, 0.01]  # Beta values
states = [-1, 1]  # Possible spin states

# Function to calculate the interaction energy between two nodes
def interaction_energy(s1, s2, beta):
    return np.exp(beta * s1 * s2)

# Function to calculate the joint probability of x_{1,10} and x_{10,10}
def joint_probability(beta, N, states):
    # Initialize all possible configurations for one column
    column_states = list(product(states, repeat=N))
    num_states = len(column_states)
    
    # Calculate interaction potentials for all combinations of adjacent column states
    potentials = np.zeros((num_states, num_states))
    for i, col1 in enumerate(column_states):
        for j, col2 in enumerate(column_states):
            potentials[i, j] = np.product(interaction_energy(np.array(col1), np.array(col2), beta))
    
    # Initialize messages (initial probabilities) as uniform
    messages = np.ones(num_states)
    
    # Perform message passing to calculate marginal probabilities
    Z_local = np.ones(N-1)  # Array to store local partition functions
    for i in range(N-1):  # Pass messages through N-1 intermediate nodes
        messages = potentials.dot(messages)
        Z_local[i] = messages.sum()  # Calculate local partition function
        messages /= Z_local[i]  # Normalize the messages
    
    # Calculate Z as the product of local partition functions
    Z = np.product(Z_local)
    
    # Calculate joint probability for x_{1,10} and x_{10,10}
    joint_probs = np.outer(messages, messages) * potentials
    joint_probs /= joint_probs.sum()  # Normalize the joint probability
    
    # Extract probabilities for specific combinations of x_{1,10} and x_{10,10}
    prob_combinations = {
        (-1, -1): joint_probs[0, 0],
        (-1, 1): joint_probs[0, -1],
        (1, -1): joint_probs[-1, 0],
        (1, 1): joint_probs[-1, -1]
    }
    return prob_combinations

# Calculate and display probabilities for each beta value
for beta in beta_values:
    probs = joint_probability(beta, N, states)
    print(f"For beta = {beta}:")
    for (s1, s10), prob in probs.items():
        print(f"Probability of x1,10 = {s1} and x10,10 = {s10}: {prob}")
    print()


C:\Users\rache\AppData\Local\Temp\ipykernel_27048\2725918909.py:53: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  probs = joint_probability(beta, N, states)


For beta = 4:
Probability of x1,10 = -1 and x10,10 = -1: 0.0009732925340423237
Probability of x1,10 = -1 and x10,10 = 1: 1.7566483808458602e-38
Probability of x1,10 = 1 and x10,10 = -1: 1.7566483808458602e-38
Probability of x1,10 = 1 and x10,10 = 1: 0.0009732925340423141

For beta = 3:
Probability of x1,10 = -1 and x10,10 = -1: 0.0009526827012922457
Probability of x1,10 = -1 and x10,10 = 1: 8.342176327300286e-30
Probability of x1,10 = 1 and x10,10 = -1: 8.342176327300286e-30
Probability of x1,10 = 1 and x10,10 = 1: 0.0009526827012922314

For beta = 1:
Probability of x1,10 = -1 and x10,10 = -1: 0.0002744471311724596
Probability of x1,10 = -1 and x10,10 = 1: 5.656776985839809e-13
Probability of x1,10 = 1 and x10,10 = -1: 5.656776985839809e-13
Probability of x1,10 = 1 and x10,10 = 1: 0.0002744471311724551

For beta = 0.5:
Probability of x1,10 = -1 and x10,10 = -1: 4.258158475989125e-05
Probability of x1,10 = -1 and x10,10 = 1: 1.9332009572743543e-09
Probability of x1,10 = 1 and x10,10 = -